In [1]:
import QuantumRingsLib 
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit 
from QuantumRingsLib import QuantumRingsProvider, job_monitor 
from QuantumRingsLib import JobStatus 
import numpy as np 
import math 
from fractions import Fraction 
 
def gcd(a, b): 
    """Compute the greatest common divisor (GCD) using Euclidean algorithm.""" 
    while b: 
        a, b = b, a % b 
    return a 
 
def qft(qc, q, n): 
    """Applies the Quantum Fourier Transform to the quantum register.""" 
    for i in range(n): 
        qc.h(q[i]) 
        for j in range(i + 1, n): 
            qc.cu1(np.pi / float(2 ** (j - i)), q[j], q[i]) 
 
def iqft(qc, q, n): 
    """Implements the Inverse Quantum Fourier Transform (IQFT).""" 
    for i in range(n - 1, -1, -1): 
        qc.h(q[i]) 
        for j in range(i): 
            qc.cu1(-np.pi / float(2 ** (i - j)), q[j], q[i]) 
 
def quantum_period_finding(N, a): 
    """Implements quantum phase estimation to find period r of a^r mod N.""" 
    num_qubits = int(math.log2(N)) + 2  # Enough qubits for accuracy 
    q = QuantumRegister(num_qubits, 'q') 
    c = ClassicalRegister(num_qubits - 1, 'c') 
    qc = QuantumCircuit(q, c) 
     
    # Apply Hadamard to the first qubits 
    for i in range(num_qubits - 1): 
        qc.h(q[i]) 
 # Modular exponentiation: simulate a^x mod N (simplified here, should be more robust) 
    qc.x(q[num_qubits - 1]) 
    qc.barrier() 
     
    # Apply QFT instead of just inverse QFT 
    qft(qc, q, num_qubits - 1) 
     
    # Perform Inverse QFT to extract period r 
    iqft(qc, q, num_qubits - 1) 
     
    # Measure all qubits except the last one 
    for i in range(num_qubits - 1): 
        qc.measure(q[i], c[i]) 
     
    return qc 
 
def run_shors(N, a): 
    """Runs Shor’s Algorithm on Quantum Rings to factor N.""" 
    provider = QuantumRingsProvider( 
        token='rings-200.sjCanK2skqL4YIzImD4QLXGtqRZuXAzR', 
        name='jessicaryuzaki@gmail.com' 
    ) 
    backend = provider.get_backend("scarlet_quantum_rings") 
     
    # Build quantum circuit 
    qc = quantum_period_finding(N, a) 
     
    # Execute the circuit 
    job = backend.run(qc, shots=1024) 
    job_monitor(job) 
    result = job.result() 
    counts = result.get_counts() 
  # Extract the most probable measurement outcome 
    measured_value = max(counts, key=counts.get)  # Binary string result 
    decimal_value = int(measured_value, 2) 
     
    # Convert to fraction to estimate period r 
    r = Fraction(decimal_value, 2**(qc.num_qubits - 1)).denominator 
    print(f"Estimated period r: {r}") 
     
    if r % 2 == 1: 
        print("Odd period found, retry with a different a") 
        return None 
     
    # Classical post-processing 
    factor1 = gcd(N, pow(a, r // 2, N) - 1) 
    factor2 = N // factor1 
     
    if factor1 * factor2 == N: 
        print(f"Factors of {N}: {factor1}, {factor2}") 
        return factor1, factor2 
    else: 
        print("Failed to factorize, retry with different a") 
        return None 
 
def factorize_number(N, a=None):
    if a is None:
        a = 2  # Default guess value
    factors = run_shors(N, a)
    if factors:
        return list(factors)
    else:
        return []

# Example usage
N = 15
a = 7
factors = factorize_number(N, a)
print(f"Factors of {N}: {factors}")


In [19]:
# Example run (Factorizing a semiprime N = 143 using base 7) 
N = 9
i = 3
flag = False

while flag == False:
    if i==1:
        print(False)
        break
    factors = run_shors(N=N, a=i)
    if N not in factors or -N not in factors:
        print(factors)
        flag = True
    else:
        i -= 1

    # Replace with different semiprimes and a values to test
  # Replace with different semiprimes and a values to test 

 

Job Queued
Job Done.
Ending Job Monitor
Estimated period r: 2
Factors of 9: 1, 9
(1, 9)


In [ ]:
list = [4, 6, 9, 10, 14, 15, 21, 22, 25, 26, 33, 34, 35, 38, 39, 46, 49, 51, 55, 57, 58, 62, 65, 69, 74, 77, 82, 85, 86, 87, 91, 93, 94, 95, 106, 111, 115, 118, 119, 121, 122, 123, 129, 133, 134, 141, 142, 143, 145, 146, 155, 158, 159, 161, 166, 169, 177, 178, 183, 185, 187, 194, 201, 202, 203, 205, 206, 209, 213, 214, 215, 217, 218, 219, 221, 226, 235, 237, 247, 249, 253, 254, 259, 262, 265, 267, 274, 278, 287, 289, 291, 295, 298, 299, 301, 302, 303, 305, 309, 314, 319, 321, 323, 326, 327, 329, 334, 335, 339, 341, 346, 355, 358, 361, 362, 365, 371, 377, 381, 382, 386, 391, 393, 394, 395, 398, 403, 407, 411, 413, 415, 417, 422, 427, 437, 445, 446, 447, 451, 453, 454, 458, 466, 469, 471, 473, 478, 481, 482, 485, 489, 493, 497, 501, 502, 505, 511, 514, 515, 517, 519, 526, 527, 529, 533, 535, 537, 538, 542, 543, 545, 551, 553, 554, 559, 562, 565, 566, 573, 579, 581, 583, 586, 589, 591, 597, 611, 614, 622, 623, 626, 629, 633, 634, 635, 649, 655, 662, 667, 669, 671, 674, 679, 681, 685, 687, 689, 694, 695, 697, 698, 699, 703, 706, 707, 713, 717, 718, 721, 723, 731, 734, 737, 745, 746, 749, 753, 755, 758, 763, 766, 767, 771, 778, 779, 781, 785, 789, 791, 793, 794, 799, 802, 803, 807, 813, 815, 817, 818, 831, 835, 838, 841, 842, 843, 849, 851, 862, 865, 866, 869, 871, 878, 879, 886, 889, 893, 895, 898, 899, 901, 905, 913, 914, 917, 921, 922, 923, 926, 933, 934, 939, 943, 949, 951, 955, 958, 959, 961, 965, 973, 974, 979, 982, 985, 989, 993, 995, 998, 1003, 1006, 1007, 1011, 1018, 1027, 1037, 1041, 1042, 1043, 1046, 1047, 1055, 1057, 1059, 1067, 1073, 1077, 1079, 1081, 1082, 1094, 1099, 1101, 1111, 1114, 1115, 1119, 1121, 1126, 1133, 1135, 1137, 1138, 1139, 1141, 1142, 1145, 1147, 1149, 1154, 1157, 1159, 1165, 1167, 1169, 1174, 1177, 1186, 1189, 1191, 1195, 1198, 1199, 1202, 1203, 1205, 1207, 1211, 1214, 1219, 1226, 1227, 1234, 1238, 1241, 1243, 1247, 1253, 1255, 1257, 1261, 1262, 1263, 1267, 1271, 1273, 1282, 1285, 1286, 1293, 1294, 1299, 1306, 1313, 1315, 1317, 1318, 1322, 1329, 1333, 1337, 1339, 1343, 1345, 1346, 1347, 1349, 1351, 1354, 1355, 1357, 1363, 1366, 1369, 1371, 1379, 1382, 1383, 1385, 1387, 1389, 1391, 1393, 1397, 1401, 1402, 1403, 1405, 1411, 1415, 1417, 1418, 1437, 1438, 1441, 1454, 1457, 1461, 1465, 1466, 1469, 1473, 1477, 1478, 1486, 1497, 1501, 1502, 1507, 1509, 1513, 1514, 1517, 1522, 1527, 1529, 1535, 1537, 1538, 1541, 1546, 1555, 1561, 1563, 1565, 1569, 1574, 1577, 1585, 1589, 1591, 1594, 1603, 1618, 1622, 1623, 1631, 1633, 1639, 1641, 1642, 1643, 1646, 1649, 1651, 1654, 1655, 1658, 1661, 1671, 1673, 1678, 1679, 1681, 1685, 1687, 1689, 1691, 1703, 1706, 1707, 1711, 1713, 1714, 1717, 1718, 1726, 1727, 1731, 1735, 1739, 1745, 1751, 1754, 1757, 1761, 1762, 1763, 1765, 1766, 1769, 1774, 1779, 1781, 1793, 1795, 1797, 1799, 1803, 1807, 1814, 1817, 1819, 1821, 1822, 1829, 1835, 1837, 1838, 1839, 1841, 1843, 1849, 1851, 1853, 1857, 1858, 1865, 1874, 1882, 1883, 1891, 1893, 1894, 1895, 1897, 1903, 1906, 1909, 1915, 1919, 1921, 1923, 1927, 1929, 1934, 1937, 1939, 1941, 1942, 1943, 1945, 1954, 1957, 1959, 1961, 1963, 1966, 1967, 1969, 1977, 1981, 1982, 1983, 1985, 1991, 1994, 2005, 2018, 2019, 2021, 2026, 2031, 2033, 2038, 2041, 2042, 2045, 2047, 2049, 2051, 2059, 2062, 2066, 2071, 2073, 2077, 2078, 2095, 2098, 2101, 2102, 2103, 2105, 2117, 2119, 2122, 2123, 2126, 2127, 2138, 2147, 2149, 2155, 2157, 2159, 2165, 2167, 2171, 2173, 2174, 2177, 2181, 2182, 2183, 2186, 2189, 2191, 2194, 2195, 2199, 2201, 2206, 2209, 2215, 2217, 2218, 2219, 2227, 2229, 2231, 2234, 2245, 2246, 2249, 2253, 2257, 2258, 2263, 2271, 2279, 2283, 2285, 2291, 2302, 2305, 2306, 2307, 2315, 2317, 2319, 2321, 2323, 2326, 2327, 2329, 2335, 2342, 2353, 2359, 2361, 2362, 2363, 2369, 2374, 2386, 2391, 2395, 2402, 2407, 2413, 2419, 2426, 2427, 2429, 2433, 2434, 2435, 2443, 2446, 2449, 2453, 2455, 2458, 2461, 2462, 2463, 2469, 2471, 2474, 2479, 2481, 2483, 2487, 2489, 2491, 2495, 2497, 2498, 2501, 2507, 2509, 2513, 2515, 2517, 2518, 2519, 2533, 2537, 2545, 2554, 2558, 2559, 2561, 2563, 2566, 2567, 2569, 2571, 2573, 2577, 2578, 2581, 2582, 2587, 2589, 2594, 2599, 2602, 2603, 2605, 2606, 2611, 2614, 2615, 2623, 2627, 2629, 2631, 2638, 2641, 2642, 2643, 2649, 2651, 2653, 2654, 2661, 2669, 2681, 2701, 2705, 2721, 2722, 2723, 2733, 2734, 2735, 2743, 2746, 2747, 2757, 2759, 2761, 2762, 2771, 2773, 2779, 2785, 2787, 2798, 2807, 2809, 2811, 2813, 2815, 2818, 2823, 2827, 2831, 2839, 2841, 2845, 2846, 2854, 2855, 2858, 2859, 2863, 2866, 2867, 2869, 2878, 2881, 2885, 2893, 2894, 2899, 2901, 2902, 2906, 2911, 2913, 2918, 2921, 2923, 2929, 2931, 2933, 2935, 2941, 2942, 2947, 2949, 2951, 2959, 2962, 2965, 2966, 2973, 2974, 2977, 2978, 2981, 2983, 2986, 2987, 2991, 2993, 2995, 2998, 3005, 3007, 3013, 3017, 3022, 3027, 3029, 3031, 3035, 3039, 3043, 3046, 3047, 3053, 3057, 3062, 3063, 3065, 3071, 3073, 3077, 3085, 3086, 3091, 3093, 3095, 3097, 3098, 3099, 3101, 3103, 3106, 3107, 3113, 3117, 3118, 3127, 3131, 3133, 3134, 3139, 3142, 3143, 3147, 3149, 3151, 3153, 3155, 3158, 3161, 3166, 3173, 3183, 3189, 3193, 3194, 3197, 3199, 3202, 3205, 3207, 3214, 3215, 3218, 3223, 3226, 3227, 3233, 3235, 3238, 3239, 3241, 3242, 3247, 3254, 3261, 3263, 3265, 3269, 3273, 3274, 3277, 3279, 3281, 3287, 3291, 3293, 3295, 3305, 3309, 3314, 3317, 3326, 3327, 3334, 3337, 3338, 3341, 3349, 3351, 3353, 3365, 3369, 3377, 3379, 3383, 3385, 3386, 3387, 3394, 3397, 3398, 3401, 3403, 3409, 3415, 3418, 3419, 3421, 3427, 3431, 3437, 3439, 3442, 3443, 3446, 3453, 3455, 3459, 3466, 3473, 3481, 3482, 3487, 3489, 3493, 3494, 3497, 3503, 3505, 3506, 3513, 3518, 3521, 3523, 3543, 3545, 3551, 3554, 3561, 3563, 3566, 3569, 3574, 3578, 3579, 3587, 3589, 3595]
working = []
not_working = []
for i in list:
    factors = run_shors(N=i, a=math.floor(math.sqrt(i)))  # Replace with different semiprimes and a values to test
    if i not in factors:
        working.append(i)  
    else:
        not_working.append(i)

print(working)

In [23]:
run_shors(N=15, a=7)

Job Queued
Job Done.
Ending Job Monitor
Estimated period r: 16
Factors of 15: 5, 3


(5, 3)

In [1]:
pip install quantumrings-toolkit-qiskit